# Projeto 2 - Ciência dos Dados


Nome: Bruno Meneghesso da Silva

Nome: Diogo Nobre de Araujo Cintra

___
# Classificador automático de sentimento para o Twitter


### Importando as bibliotecas

In [2]:
%%capture
!pip install tweepy
%matplotlib inline
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
from numpy import arange
from numpy import percentile
import numpy as np
import os
import re 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

### Autenticando aconta do Twitter

#### Conta: @Ciencia dos dados

#### Coletando os dados do Twitter:

#### Salvando os dados em uma planilha Excel:

### Classificação dos tweets
#### 0 = irrelevante
#### 1 = relevante

#### Abrindo a planilha do Excel

In [3]:
TREINAMENTO = pd.read_excel('Correios.xlsx',sheet_name = 'Treinamento')
TESTE = pd.read_excel('Correios.xlsx',sheet_name = 'Teste')

#### Filtrando base de treinamento em relevante e irrelevante

In [4]:
TREINAMENTO_RELEVANTE = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==1]
TREINAMENTO_IRRELEVANTE = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==0]

#### Função para limpar a base de dados

In [5]:
def clean(text):
    punctuation = '[!\-.:?;#$%&*_1234567890"]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

#### Aplicando a função e limpando a base de dados

In [6]:
texto_relevante = " ".join(TREINAMENTO_RELEVANTE['Treinamento'])
texto_relevante_1 = clean(texto_relevante)
texto_relevante_2 = texto_relevante_1.split()
texto_relevante_3 = []

texto_irrelevante = " ".join(TREINAMENTO_IRRELEVANTE['Treinamento'])
texto_irrelevante_1 = clean(texto_irrelevante)
texto_irrelevante_2 = texto_irrelevante_1.split()
texto_irrelevante_3 = []

treinamento = " ".join(TREINAMENTO['Treinamento'])
treinamento_1 = clean(treinamento)
treinamento_2 = treinamento_1.split()
treinamento_3 = []

teste = " ".join(TESTE['Teste'])
teste_1 = clean(teste)
teste_2 = teste_1.split()
teste_3 = []

for e in range(len(texto_relevante_2)-1):
    if texto_relevante_2[e] != 'rt' and texto_relevante_2[e][0] != '@':     
        texto_relevante_3.append(texto_relevante_2[e])    
        
for e in range(len(texto_irrelevante_2)-1):
    if texto_irrelevante_2[e] != 'rt' and texto_irrelevante_2[e][0] != '@':     
        texto_irrelevante_3.append(texto_irrelevante_2[e]) 
        
for e in range(len(treinamento_2)-1):
    if treinamento_2[e] != 'rt' and treinamento_2[e][0] != '@':     
        treinamento_3.append(treinamento_2[e])
        
for e in range(len(teste_2)-1):
    if teste_2[e] != 'rt' and teste_2[e][0] != '@':     
        teste_3.append(teste_2[e])        
        
texto_relevante_4 = pd.Series(texto_relevante_3)
texto_irrelevante_4 = pd.Series(texto_irrelevante_3)
treinamento_4 = pd.Series(treinamento_3)
teste_4 = pd.Series(teste_3)

total_amostral_teste = len(teste_4)
total_amostral_treinamento = len(treinamento_4)
total_relevante = texto_relevante_4.value_counts()
total_irrelevante = texto_irrelevante_4.value_counts()

#### Probabilidade do tweet ser relevante

In [7]:
prob_relevante = len(TREINAMENTO.loc[TREINAMENTO["Classificacao"]==1])/len(TREINAMENTO["Treinamento"])
print("Probabilidade de um tweet ser relevante: {:.2f}%".format(prob_relevante*100))

Probabilidade de um tweet ser relevante: 40.13%


#### Probabilidade do tweet ser relevante

In [8]:
prob_irrelevante = len(TREINAMENTO.loc[TREINAMENTO["Classificacao"]==0])/len(TREINAMENTO["Treinamento"])
print("Probabilidade de um tweet ser relevante: {:.2f}%".format(prob_irrelevante*100))

Probabilidade de um tweet ser relevante: 59.87%


#### Probabilidade de uma palavra ser relevante

In [9]:
prob_relevante_palavra = total_relevante/total_amostral_treinamento
#print("Probabilidade de cada palavra ser relevante")
#print(prob_relevante_palavra*100)

#### Probabilidade de uma palavra ser irrelevante

In [10]:
prob_irrelevante_palavra = total_irrelevante/total_amostral_treinamento
#print("Probabilidade de cada palavra ser irrelevante")
#print(prob_irrelevante_palavra*100)

### Montando o Classificador Naive-Bayes

#### Funções para o cálculo da probabilidade de ser relevante e irrelevante

In [11]:
prob_relevante_teste = len(TESTE.loc[TESTE["Classificacao"]==1])/len(TESTE["Teste"])
prob_irrelevante_teste = len(TESTE.loc[TESTE["Classificacao"]==0])/len(TESTE["Teste"])

smoth = 1e-10 #é uma estimativa e deve mudar
vocabulario = 356000

# defini probabilidade de ser relevante:
def relevante(tweet):
    
    tweet_relevante_2 = clean(tweet)
    tweet_relevante_3 = tweet_relevante_2.split()
    tweet_relevante_4 = []
    
    for e in range(len(tweet_relevante_3)-1):
        
        if tweet_relevante_3[e] != 'rt' and tweet_relevante_3[e][0] != '@':     
            tweet_relevante_4.append(tweet_relevante_3[e])
            
    prob = 1
    
    for p in tweet_relevante_4:
        
        if p in prob_relevante_palavra:
            prob = prob*(prob_relevante_palavra[p]*total_relevante.size+smoth)/(total_relevante.size+smoth*vocabulario)*1000
        else:
            prob = prob*(smoth)/(total_relevante.size+smoth*vocabulario)*1000
            
    prob = prob * prob_relevante
    prob = math.log10(prob)
    return prob

# defini probabilidade de ser irrelevante:
def irrelevante(tweet):
    
    tweet_irrelevante_2 = clean(tweet)
    tweet_irrelevante_3 = tweet_irrelevante_2.split()
    tweet_irrelevante_4 = []
    
    for e in range(len(tweet_irrelevante_3)-1):
        
        if tweet_irrelevante_3[e] != 'rt' and tweet_irrelevante_3[e][0] != '@':     
            tweet_irrelevante_4.append(tweet_irrelevante_3[e])
            
    prob = 1
    
    for p in tweet_irrelevante_4:
        
        if p in prob_irrelevante_palavra:
            prob = prob*(prob_irrelevante_palavra[p]*total_irrelevante.size+smoth)/(total_irrelevante.size+smoth*vocabulario)*1000
        else:
            prob = prob*(smoth)/(total_irrelevante.size+smoth*vocabulario)*1000
            
    prob = prob * prob_irrelevante
    prob = math.log10(prob)
    return prob

#### Verificando a performance do algoritmo com a base de teste

In [25]:
def compara (tweet):
    
    r = relevante(tweet)
    nor = irrelevante(tweet)
    
    return r>nor

rev = 0
norev = 0

#rev = Relevante
#norev = Irrelevante

relevantes_algoritimo = []

for e in TESTE["Teste"]:
    
    if compara(e):
        rev +=1
        relevantes_algoritimo.append(1)
    else:
        norev+=1
        relevantes_algoritimo.append(0)
        
VP = 0 #Verdadeiros Positivos
FP = 0 #Falsos Positivos
VN = 0 #Verdadeiros Negativos
FN = 0 #Falsos Negativos

for n in range (len(relevantes_algoritimo)-1):
    
    if TESTE["Classificacao"][n] == relevantes_algoritimo[n]:
        if relevantes_algoritimo[n] == 1:
            VP+=1
        else:
            FP+=1
    else:
        if relevantes_algoritimo[n] == 1:
            VN+=1
        else:
            FN+=1

### Resultados do algoritmo

In [26]:
print("Probabilidade de ser relevante segundo o algoritmo: {:.2f}%".format((rev/TESTE["Teste"].size)*100))
print("Probabilidade de ser irrelevante segundo o algoritmo: {:.2f}%".format((norev/TESTE["Teste"].size)*100))
print("Probabilidade de ser relevante pela nossa classificação: {:.2f}%".format(prob_relevante_teste*100))
print("Probabilidade de ser irrelevante pela nossa classificação: {:.2f}%".format(prob_irrelevante_teste*100))
print("")
print("Porcentagem de Verdadeiros Positivos: {:.2f}%".format(VP/TESTE["Classificacao"].size*100))
print("Porcentagem de Falsos Positivos: {:.2f}%".format(FP/TESTE["Classificacao"].size*100))
print("Porcentagem de Verdadeiros Negativos: {:.2f}%".format(VN/TESTE["Classificacao"].size*100))
print("Porcentagem de Falsos Negativos: {:.2f}%".format(FN/TESTE["Classificacao"].size*100))
print("")
print("Acurácia do algoritmo: {:.2f}%".format((rev/TESTE["Teste"].size/prob_relevante_teste)*100))

Probabilidade de ser relevante segundo o algoritmo: 28.50%
Probabilidade de ser irrelevante segundo o algoritmo: 71.50%
Probabilidade de ser relevante pela nossa classificação: 44.50%
Probabilidade de ser irrelevante pela nossa classificação: 55.50%

Porcentagem de Verdadeiros Positivos: 18.50%
Porcentagem de Falsos Positivos: 45.50%
Porcentagem de Verdadeiros Negativos: 10.00%
Porcentagem de Falsos Negativos: 25.50%

Acurácia do algoritmo: 64.04%


___
### Conclusão
#### Validação da acurácia | Com o auxílio do Professor Fábio José Ayres 

In [27]:
Xt = TREINAMENTO['Treinamento']
y = TREINAMENTO['Classificacao']

cnt = CountVectorizer()
X = cnt.fit_transform(Xt)
nb = MultinomialNB(alpha=1e-10)
nb.fit(X, y)

Xt_test = TESTE['Teste']
y_test = TESTE['Classificacao']

X_test = cnt.transform(Xt_test)
ypred = nb.predict(X_test)

acuracia_biblioteca = accuracy_score(y_test, ypred)*100

print("Acurácia do resultado segundo a biblioteca: {:.2f}%".format(acuracia_biblioteca))

Acurácia do resultado segundo a biblioteca: 64.50%


A acurácia do algoritmo desenvolvido pelos integrantes do grupo, foi de 64.04% com o valor do *smoth* de 1e-10. Para validar a acurácia encontrada, com a ajuda do professor Fábio José Ayres, foi utilizado recursos das bibliotecas “sklearn.feature_extraction.text”, “sklearn.naive_bayes” e “sklearn.metrics” para encontrar o valor da acurácia de um algoritmo presente nas bibliotecas que utilizam o classificador de Naive-Bayes. O código acima calcula o valor da acurácia utilizando os recursos das bibliotecas já mencionadas, o valor obtido foi de uma acurácia igual a 64.50%. Dessa forma conclui-se que o algoritmo desenvolvido obteve resultados acima do esperado, com apenas uma diferença para menos da acurácia de 0.46%.

___
### Aperfeiçoamento:

Para melhorar a acurácia do algoritmo, deveríamos ter três bases de dados: Treinamento, Teste, Controle. Dessa forma iriamos treinar o algoritmo com a base de treinamento, depois testar com a base de teste e variar o valor do *smoth* até que a acurácia do algoritmo seja a melhor possível. Após achar o valor do *smoth* com a base de teste, aplicaríamos o algoritmo na base de controle, observando o valor da acurácia.

Além disso a probabilidade base d um tweet ser relevante poderia ser alterada com o intuito de melhorar a acuracia, evitando que relevantes sejam considerados irrelevantes, porém, com os recursos hoje possuidos pelos aunos não é possivel argumentar para um determinado ajuste. O unico método possivel seria manipular diretamente o valor e reiterar com ele para atingir o valor desejado, atitude que não seria condizente com a intenção da atividade.

### Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) 